### Install Kalshi - python

In [ ]:
pip install kalshi-python

You should consider upgrading via the 'c:\Users\arune\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


#### Create an .env file and add yoru credentials 
- DEMO_EMAIL=your_email
- DEMO_PASSWORD=your_password

In [ ]:
pip install python-dotenv


In [1]:
# Welcome to the Kalshi REST v2 Starter Code!

# pypi client: recommended for more advanced programmers
import kalshi_python
from pprint import pprint

# starter client: recommended for all levels of programming experience (what this client is implemented using)
from KalshiClientsBaseV2 import ExchangeClient
import time
import json
import uuid
from kalshi_python.models import *
from datetime import datetime

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

demo_email = os.getenv("DEMO_EMAIL")
demo_password = os.getenv("DEMO_PASSWORD")

# for prod
# prod_api_base = "https://trading-api.kalshi.com/trade-api/v2"

# for demo
demo_api_base = "https://demo-api.kalshi.co/trade-api/v2"

## if wanting to test in demo
exchange_client = ExchangeClient(
    exchange_api_base=demo_api_base,
    email=demo_email,
    password=demo_password
)

# Check the exchange status to confirm the connection
print(exchange_client.get_exchange_status())


In [ ]:
import os
from dotenv import load_dotenv
import kalshi_python

# Load environment variables from .env file
load_dotenv()

email = os.getenv("KALSHI_EMAIL")
password = os.getenv("KALSHI_PASSWORD")

config = kalshi_python.Configuration()
# Comment the line below to use production
config.host = 'https://demo-api.kalshi.co/trade-api/v2'

# Create an API configuration passing your credentials.
kalshi_api = kalshi_python.ApiInstance(
    email=email,
    password=password,
    configuration=config,
)

# Checks if the exchange is available.
exchangeStatus = kalshi_api.get_exchange_status()
print('Exchange status response: ')
print(exchangeStatus)


Exchange status response: 
{'exchange_active': True, 'trading_active': True}


In [4]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('predicted_temperature_max.csv')
print(df)
# # Get the predicted temperature for Austin
# predicted_temp_for_austin = df.loc[df['Location'] == 'Austin', 'predicted_tempmax'].values[0]
# print(f"Predicted Temperature for Austin: {predicted_temp_for_austin}")


   Location  predicted_tempmax
0  New York               49.0
1   Chicago               51.0
2    Austin               75.0
3     Miami               84.0


In [5]:
def determine_market_ticker_from_response(eventResponse, predicted_max_temp):
    for market in eventResponse.markets:
        subtitle = market.subtitle  # Adjusted based on your object structure
        if 'or below' in subtitle:
            max_temp = int(subtitle.split('°')[0])
            if predicted_max_temp <= max_temp:
                return market.ticker.split('-')[-1]
        elif 'or above' in subtitle:
            min_temp = int(subtitle.split('°')[0]) + 1
            if predicted_max_temp >= min_temp:
                return market.ticker.split('-')[-1]
        else:
            # Handling ranges correctly by cleaning the string of non-numeric before conversion
            temp_range = [int(temp.split('°')[0]) for temp in subtitle.split(' to ')]
            if temp_range[0] <= predicted_max_temp <= temp_range[1]:
                return market.ticker.split('-')[-1]
    return None


In [11]:
seriesTickers = {
    'New York': 'HIGHNY',
    'Chicago': 'HIGHCHI',
    'Austin': 'HIGHAUS',
    'Miami': 'HIGHMIA'
}
today = datetime.today()

# print(f"Processing cities: {df['Location'].tolist()}")

# for index, row in df.iterrows():
#     city = row['Location']
#     print(f"Processing {city}...")

# # Format the date as 'YYMMDD'
date_str = today.strftime('%y%b%d').upper()
for index, row in df.iterrows():
    city = row['Location']
    predicted_max_temp = row['predicted_tempmax']
    seriesTicker = seriesTickers[city]
    

    eventTicker = f'{seriesTicker}-{date_str}'
    eventResponse = kalshi_api.get_event(eventTicker)  # Replace with actual API call
    
    # Parse the event response to find matching market ticker
    # This code block assumes a function to parse and match the market ticker based on predicted temperature
    matching_market_ticker = determine_market_ticker_from_response(eventResponse, predicted_max_temp)
    
    # Now you have the matching market ticker, you can retrieve the order book
    marketTicker = f'{eventTicker}-{matching_market_ticker}'
    orderbookResponse = kalshi_api.get_market_orderbook(marketTicker)  # Replace with actual API call
    
    print(f'\nOrderbook for market: {marketTicker}')
    # Assuming pprint is imported or defined
    pprint(orderbookResponse)
# print(dir(orderbookResponse))



Orderbook for market: HIGHNY-24APR03-B48.5
{'orderbook': {'no': [], 'yes': [[99, 1]]}}

Orderbook for market: HIGHCHI-24APR03-T42
{'orderbook': {'no': [], 'yes': [[25, 50], [50, 20], [99, 1]]}}

Orderbook for market: HIGHAUS-24APR03-B74.5
{'orderbook': {'no': [], 'yes': [[99, 1]]}}

Orderbook for market: HIGHMIA-24APR03-B84.5
{'orderbook': {'no': [], 'yes': [[30, 1000], [99, 1]]}}


In [ ]:
yes_offers = orderbookResponse.orderbook.yes
orderbook_dict = orderbookResponse.to_dict()
yes_offers = orderbook_dict['orderbook']['yes']


In [16]:
orderbook_dict = orderbookResponse.to_dict()
yes_offers = orderbook_dict['orderbook']['yes']
for index, row in df.iterrows():
    city = row['Location']
    predicted_max_temp = row['predicted_tempmax']
    seriesTicker = seriesTickers[city]

    eventTicker = f'{seriesTicker}-{date_str}'
    eventResponse = kalshi_api.get_event(eventTicker)

    matching_market_ticker = determine_market_ticker_from_response(eventResponse, predicted_max_temp)
    
    if matching_market_ticker:
        marketTicker = f'{eventTicker}-{matching_market_ticker}'
        orderbookResponse = kalshi_api.get_market_orderbook(marketTicker)
        
        # Convert the orderbookResponse to a dictionary to access 'yes' and 'no' offers
        orderbook_dict = orderbookResponse.to_dict()
        yes_offers = orderbook_dict['orderbook']['yes'] if 'yes' in orderbook_dict['orderbook'] else []
        
        if yes_offers:
            best_yes_offer = min([offer[0] for offer in yes_offers])
            
            if exchangeStatus.trading_active:
                orderUuid = str(uuid.uuid4())
                orderResponse = kalshi_api.create_order(CreateOrderRequest(       
                    ticker=marketTicker,
                    action='buy',
                    type='limit',
                    yes_price=best_yes_offer - 1,  # Adjust based on your strategy
                    count=1000,  # And your desired amount
                    client_order_id=orderUuid,
                    side='yes',
                ))
                print(f'\nOrder submitted for {city}: ')
                pprint(orderResponse)
        else:
            print(f'\nNo "yes" offers available for {city}, no orders will be placed.')
    else:
        print(f'No matching market ticker found for {city} based on the predicted temperature.')
 


Order submitted for New York: 
{'order': {'action': 'buy',
           'client_order_id': '473a0902-560a-4bf6-972c-6983d1903f62',
           'created_time': '2024-04-03T15:05:17.307775Z',
           'expiration_time': None,
           'no_price': 2,
           'order_id': '60575f25-f4be-4ef1-b441-b66b12eafd5c',
           'side': 'yes',
           'status': 'resting',
           'ticker': 'HIGHNY-24APR03-B48.5',
           'type': 'limit',
           'user_id': '8eb92856-b0cd-4c17-bc68-637fcde6ecfe',
           'yes_price': 98}}

Order submitted for Chicago: 
{'order': {'action': 'buy',
           'client_order_id': 'd5a4dc45-977a-48df-8344-7875518ceafa',
           'created_time': '2024-04-03T15:05:17.713731Z',
           'expiration_time': None,
           'no_price': 76,
           'order_id': '6d6ecf7d-c09e-429b-85e4-e08d4c86631d',
           'side': 'yes',
           'status': 'resting',
           'ticker': 'HIGHCHI-24APR03-T42',
           'type': 'limit',
           'user_id': 